In [1]:
import os
import google.generativeai as genai
import faiss
import numpy as np
import pickle
import json
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision,context_recall

In [ ]:
# LLM API 연동 테스트
genai.configure(api_key="")
model = genai.GenerativeModel('gemini-2.0-flash')
response = model.generate_content("인공지능에 대해 한 문장으로 설명하세요.")

In [3]:
# 주요 기능 함수

def create_faiss_index(texts, model, save_path="faiss_index.pkl"):
    """텍스트 리스트를 임베딩 후 FAISS에 저장 및 저장 파일 생성"""
    embeddings = model.encode(texts, convert_to_numpy=True)
    embeddings = normalize(embeddings, axis=1)  # 코사인 유사도 적용을 위해 정규화
    index = faiss.IndexFlatIP(embeddings.shape[1])  # 코사인 유사도 기반 Index
    index.add(embeddings)

    # FAISS 인덱스 및 텍스트 저장
    with open(save_path, "wb") as f:
        pickle.dump({"index": index, "texts": texts}, f)

    return index, embeddings

def load_faiss_index(load_path="faiss_index.pkl"):
    """저장된 FAISS 인덱스를 로드"""
    with open(load_path, "rb") as f:
        data = pickle.load(f)
    return data["index"], data["texts"]

def retrieve_documents(query, index, model, texts, top_k=5):
    """입력 질문과 가장 유사한 문서 청크 top_k 개 반환"""
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding, axis=1)  # 정규화
    distances, indices = index.search(query_embedding, top_k)
    retrieved_chunks = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return sorted(retrieved_chunks, key=lambda x: x[1], reverse=True)  # 유사도 순 정렬

def generate_gemini_answer(query, context):
    """Gemini API를 활용하여 답변 생성"""
    prompt = f"Context: {context} \n\nQuestion: {query}에 대해 답변을 하되, context를 요약하는 방향으로 답변 생성해줘."
    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content(prompt)
    return response.text

In [4]:
# 10개 문서 데이터 불러오기
dir_path = "/Users/masang/Desktop/aiffel/mini-aiffelton/data"

file_path_list = [
    "/BU_S0000105.json",
    "/BU_S0000286.json",
    "/BU_S0000337.json",
    "/BU_S0000473.json",
    "/BU_S0000474.json",
    "/BU_S0000565.json",
    "/BU_S0000803.json",
    "/BU_S0000865.json",
    "/BU_S0000954.json",
    "/BU_S0000957.json"
]

In [5]:
# json 파일 불러오기

all_corpus_texts = []

for i in range(10):
    file_path = str(dir_path + file_path_list[i])  # 변수명을 문자열로 접근
    with open(file_path, "r", encoding="utf-8-sig") as f:
        json_data = json.load(f)

    if "data" in json_data and isinstance(json_data["data"], list):
        # 각 요소에서 'corpus' 값 가져오기
        corpus_list = [item.get("corpus", "corpus 없음") for item in json_data["data"]]

        # 리스트를 문자열로 합쳐서 저장
        all_corpus_texts.append("\n".join(corpus_list))
    else:
        print(f"파일 {file_path}: 'data' 키가 없거나 리스트 형식이 아닙니다.")

# 모든 corpus_text를 하나의 문자열로 합치기
final_corpus_text = "\n".join(all_corpus_texts)

print("모든 Corpus 내용:")
print(final_corpus_text)

모든 Corpus 내용:
제1장 연구개요
I. 연구배경 및 목적
1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구원에서 1996년 조사모델과 방법을 개발한 이후 현재까지 지속적으로 수행하고 있는 중앙정부 및 지방정부의 과제이다. 
횡성군 민원행정서비스 만족도조사는 횡성군 종합민원실에서 제공하는 민원행정서비스를 이용한 지역주민과 이용객을 대상으로 하여 행정서비스의 전달체계와 그 서비스 과정에서의 공무원들의 행태와 민원제도 등에 대해 체감하는 만족도를 조사하고자 실시되었다. 
특히 상반기조사결과를 비교 대상으로 하여 하반기 고객 만족도의 수준을 평가하고 발전지향적인 고객중심의 행정서비스 체제를 구축하는데 있어 자료로 활용함으로써 횡성군이 제공하는 민원행정서비스의 질적 향상에 기여하고자 한다. 
▷ 친절하고 쾌적한 민원실
▷ 고객중심의 초일류 민원 행정기관으로서의 위상정립
▷ 이용고객에 대한 과학적 분석을 토대로 장기발전전략 수립
▷ 고객만족도 제고를 위한 기초조사실시의 필요성 부각
▷ 눈높이 민원행정의 실현
2. 연구의 목적
횡성군 종합민원실에서 제공하는 행정서비스의 고객인 이용자들로 하여금 민원행정서비스의 전달체계와 그 서비스 과정에서 공무원들의 행태와 민원 제도, 시설 등에 대하여 체감정도를 조사를 목적으로 하고 있다. 
횡성군 민원행정 서비스 고객만족도 조사는 구체적으로 다음과 같은 의의를 내포하고 있다. 
▷ 고객중심의 초일류 민원행정서비스의 실현
▷ 고객만족도 제고 및 고객감동 지향
▷ 불합리한 제도개선과 발전방향 제시
▷ 맞춤형 민원행정 서비스 실현
▷ 전자민원 행정구현을 위한 기초조사
▷ 행정업무 분야별 만족도 제고
▷ 행정서비스의 취약점과 개선방향을 도출
▷ 과거의 고객만족도 결과와 비교하여 고객만족도 수준의 변화정도를 분석고객만족도 조사는 궁극적으로 평가대상이 되는 종합민원실로 하여금 고객중심의 행정서비스를 구현하며, 행정서비스의 질을 향상시킴으로써 횡성군의 경쟁력 제고를 궁극적인 목표로 하고 있다고 할 수 있다. 
II. 연구범위 

In [6]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings

In [7]:
# 한국어 지원 임베딩 모델 설정
model = SentenceTransformer('jhgan/ko-sbert-sts')
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

# SemanticChunker 설정
text_splitter = SemanticChunker(embedding_model)

/var/folders/qj/sfy11gcx28d6_5n6tf2m8wc00000gn/T/ipykernel_25509/3392555332.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


In [8]:
# 문서별로 청크화
texts = text_splitter.split_text("\n\n".join(final_corpus_text.split("\n\n")))

print(texts[:5])

['제1장 연구개요\nI.', '연구배경 및 목적\n1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구원에서 1996년 조사모델과 방법을 개발한 이후 현재까지 지속적으로 수행하고 있는 중앙정부 및 지방정부의 과제이다. 횡성군 민원행정서비스 만족도조사는 횡성군 종합민원실에서 제공하는 민원행정서비스를 이용한 지역주민과 이용객을 대상으로 하여 행정서비스의 전달체계와 그 서비스 과정에서의 공무원들의 행태와 민원제도 등에 대해 체감하는 만족도를 조사하고자 실시되었다. 특히 상반기조사결과를 비교 대상으로 하여 하반기 고객 만족도의 수준을 평가하고 발전지향적인 고객중심의 행정서비스 체제를 구축하는데 있어 자료로 활용함으로써 횡성군이 제공하는 민원행정서비스의 질적 향상에 기여하고자 한다. ▷ 친절하고 쾌적한 민원실\n▷ 고객중심의 초일류 민원 행정기관으로서의 위상정립\n▷ 이용고객에 대한 과학적 분석을 토대로 장기발전전략 수립\n▷ 고객만족도 제고를 위한 기초조사실시의 필요성 부각\n▷ 눈높이 민원행정의 실현\n2. 연구의 목적\n횡성군 종합민원실에서 제공하는 행정서비스의 고객인 이용자들로 하여금 민원행정서비스의 전달체계와 그 서비스 과정에서 공무원들의 행태와 민원 제도, 시설 등에 대하여 체감정도를 조사를 목적으로 하고 있다. 횡성군 민원행정 서비스 고객만족도 조사는 구체적으로 다음과 같은 의의를 내포하고 있다. ▷ 고객중심의 초일류 민원행정서비스의 실현\n▷ 고객만족도 제고 및 고객감동 지향\n▷ 불합리한 제도개선과 발전방향 제시\n▷ 맞춤형 민원행정 서비스 실현\n▷ 전자민원 행정구현을 위한 기초조사\n▷ 행정업무 분야별 만족도 제고\n▷ 행정서비스의 취약점과 개선방향을 도출\n▷ 과거의 고객만족도 결과와 비교하여 고객만족도 수준의 변화정도를 분석고객만족도 조사는 궁극적으로 평가대상이 되는 종합민원실로 하여금 고객중심의 행정서비스를 구현하며, 행정서비스의 질을 향상시킴으로써 횡성군의 경쟁력 제고를 궁극적인 목표로 하고 있다고 할 수 있다. II.', '연구

In [9]:
# KoSBERT 임베딩 생성 및 FAISS 저장
index, embeddings = create_faiss_index(texts, model)

In [10]:
# FAISS 인덱스 로드 (저장된 인덱스를 재사용 가능)
#index, texts = load_faiss_index()

In [11]:
# 검색 성능 테스트에 쓰이는 10개 질문과 답변
queries = ["횡성군 민원행정 서비스 고객만족도 조사의 궁극적인 목표는 뭐야?",
         "횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?",
         "횡성군 종합민원실의 민원행정서비스 조사는 며칠을 선정하여 실시하였어?",
         "민원행정서비스에 대한 이용고객의 만족도를 평가하기 위한 설문지의 세부설문은 몇 개의 항목으로 구성되어 있을까?",
         "조사를 해야 하는 저임금 근로자의 범위를 정할 때 기준이 몇 개가 있어?",
         "중위임금 이하 근로자를 저임금 근로자로 선택해야 하는 이유는?",
         "사업주 및 근로자를 대상하는 조사표에서 독립질문으로 넣어야 될 문항은?",
         "지원 단체에 대한 2차 교부금은 어떤 결과에 따라 지급됐어?",
         "회계평가의 세부항목은 몇 개야?",
         "2020년 사업평가 점수가 떨어진 이유는 뭘까?"]

responses = ["종합민원실로 하여금 고객중심의 행정서비스를 구현하며, 행정서비스의 질을 향상시킴으로써 횡성군의 경쟁력을 제고하는 것입니다. ",
             "2명의 설문조사원이 투입되었습니다 ",
             "공휴일과 휴일을 제외한 10일을 선정합니다.",
             "세부설문은 6개 항목으로 구성되어 있습니다.",
             "객관성, 일관성, 목적 부합성, 조사의 용이성으로 네 가지 기준이 있습니다.",
             "저임금 근로자로 중위임금 이하 근로자를 선택해야 하는 이유는 다음과 같습니다. 첫째, 최저임금 상승을 통해 중위임금 이하 근로자들의 임금이 더 빠르게 증가되면 임금격차를 해결할 수 있기 때문입니다. 둘째, 최저임금 인상으로 중위임금 이하 근로자들의 고용이 감소할 수 있기 때문에 중위임금 이하 근로자를 저임금 근로자로 보아야 합니다. ",
             "조사표에서 응답 대상이 달라지지 않는 문항은 독립질문으로 넣어야 됩니다.",
             "답변 : 단체 역량 및 운영과정의 중간평가를 진행하였고 그 결과에 따라 2차 교부금이 지급됐습니다.",
             "답변 : 세부항목은 9개입니다.",
             "답변 : 코로나19의 영향으로 대부분의 단체들이 사업방식을 변경했는데 일정 내 계획된 단위사업을 모두 수행하지 못했기 때문입니다."]

In [12]:
q_idx = 0 # 사용할 질문 인덱스

# 쿼리 질문에 대한 문서 청크 리트리브(유사도 상위 3개)
retrieved_docs = retrieve_documents(queries[q_idx], index, model, texts, top_k=3)

# 리트리브 결과 출력
for doc, score in retrieved_docs:
    print(f"유사도: {score:.4f} | 내용: {doc[:200]}...")

# Gemini API를 활용한 답변 생성
context_text = "\n".join([doc[0] for doc in retrieved_docs])
answer = generate_gemini_answer(queries[q_idx], context_text)
print("\n🔹 Gemini 응답:", answer)

유사도: 0.7086 | 내용: 연구배경 및 목적
1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구원에서 1996년 조사모델과 방법을 개발한 이후 현재까지 지속적으로 수행하고 있는 중앙정부 및 지방정부의 과제이다. 횡성군 민원행정서비스 만족도조사는 횡성군 종합민원실에서 제공하는 민원행정서비스를 이용한 지역주민과 이용객을 대상으로 하여 행정서비스의 전달체계와 그 서비스 과정에...
유사도: 0.5247 | 내용: 11. 30(35일)
▷ 공간적 범위
- 횡성군 종합민원실
▷ 내용적 범위
- 서론(연구 개요로서 연구의 배경과 목적 및 범위 및 방법)
- 민원 만족도 조사(민원서비스 시설, 민원업무 담당공무원의 친절도, 민원처리내용 중심)
- 설문결과 분석
- 상반기 조사결과와의 비교분석
- 종합평가 및 발전방안
2. 연구방법
▷ 문헌연구 및 자료분석(literatur...
유사도: 0.4905 | 내용: 발전방안
1. 만족도 평가와 비교분석을 통한 문제점 탐색
▷ 2006년 상반기 조사결과에서 도출된 만족도 점수와 순위를 하반기 결과와 비교하여 연결한 결과 편의시설 항목이 가장 만족도가 높게 평가되었음. ▷ 각 차원에 대한 종합만족도는 세부항목간의 분석에 영향을 미칠수 있어 하반기 설문에서는 배제하였음. ▷ 민원인들이 가장 불편하게 느끼고 있는 것은 주차장...

🔹 Gemini 응답: 횡성군 민원행정 서비스 고객만족도 조사는 민원 서비스 질적 향상을 통해 고객 중심의 행정 서비스를 구현하고, 궁극적으로 횡성군의 경쟁력을 높이는 것을 목표로 한다. 이를 위해 민원 서비스 만족도 조사를 통해 불합리한 제도를 개선하고, 고객 감동을 지향하며, 맞춤형 민원 행정 서비스를 제공하고, 전자 민원 행정 구현을 위한 기초 조사를 실시하여 행정 서비스의 취약점을 개선하고자 한다.



In [22]:
# 검색된 청크 통계 확인
lengths_chars = [len(chunk) for chunk in texts]
lengths_words = [len(chunk.split()) for chunk in texts]

print("총 청크 수:", len(texts))
print("평균 글자 수:", np.mean(lengths_chars))
print("평균 단어 수:", np.mean(lengths_words))

총 청크 수: 116
평균 글자 수: 1916.25
평균 단어 수: 411.5948275862069


In [23]:
# 코사인 유사도 임계치 기준 청크 추출 함수(절대값)
'''
def retrieve_documents_by_similarity_gap(query, model, texts, threshold_gap=0.1):
    query_vec = model.encode(query, convert_to_tensor=False).tolist()
    doc_vecs = model.encode(texts, convert_to_tensor=False).tolist()

    sims = cosine_similarity([query_vec], doc_vecs)[0]
    max_sim = np.max(sims)

    # 문서와 유사도를 함께 저장
    retrieved_docs = [(texts[i], sims[i]) for i in range(len(sims)) if (max_sim - sims[i]) <= threshold_gap]

    return retrieved_docs


# 최고 유사도 0.1 하한 기준
retrieved_docs = retrieve_documents_by_similarity_gap(queries[q_idx], model, texts, threshold_gap=0.1)

# 검색 결과 출력
for doc, score in retrieved_docs:
    print(f"유사도: {score:.4f} | 내용: {doc[:200]}...")

# Gemini API를 활용한 답변 생성
context_text = "\n".join([doc[0] for doc in retrieved_docs])
answer = generate_gemini_answer(queries[q_idx], context_text)
print("\n🔹 Gemini 응답:", answer)
'''

'\ndef retrieve_documents_by_similarity_gap(query, model, texts, threshold_gap=0.1):\n    query_vec = model.encode(query, convert_to_tensor=False).tolist()\n    doc_vecs = model.encode(texts, convert_to_tensor=False).tolist()\n\n    sims = cosine_similarity([query_vec], doc_vecs)[0]\n    max_sim = np.max(sims)\n\n    # 문서와 유사도를 함께 저장\n    retrieved_docs = [(texts[i], sims[i]) for i in range(len(sims)) if (max_sim - sims[i]) <= threshold_gap]\n\n    return retrieved_docs\n\n\n# 최고 유사도 0.1 하한 기준\nretrieved_docs = retrieve_documents_by_similarity_gap(queries[q_idx], model, texts, threshold_gap=0.1)\n\n# 검색 결과 출력\nfor doc, score in retrieved_docs:\n    print(f"유사도: {score:.4f} | 내용: {doc[:200]}...")\n\n# Gemini API를 활용한 답변 생성\ncontext_text = "\n".join([doc[0] for doc in retrieved_docs])\nanswer = generate_gemini_answer(queries[q_idx], context_text)\nprint("\n🔹 Gemini 응답:", answer)\n'

In [13]:
# 코사인 유사도 임계치 기준 청크 추출 함수(비율)
def retrieve_documents_by_similarity_ratio(query, model, texts, threshold_ratio=0.1):
    query_vec = model.encode(query, convert_to_tensor=False).tolist()
    doc_vecs = model.encode(texts, convert_to_tensor=False).tolist()

    sims = cosine_similarity([query_vec], doc_vecs)[0]
    max_sim = np.max(sims)

    # 비율 선정
    threshold = max_sim * (1 - threshold_ratio)

    # 유사도 기준으로 필터링
    retrieved_docs = [(texts[i], sims[i]) for i in range(len(sims)) if sims[i] >= threshold]

    return retrieved_docs

In [14]:
# 최고 유사도 기준 30% 이내 청크 생성
results = []

for query in queries:
    retrieved_docs = retrieve_documents_by_similarity_ratio(query, model, texts, threshold_ratio=0.3)
    for doc, score in retrieved_docs:
        results.append({"Query": query, "Similarity": score, "Content": doc})  # 딕셔너리 형태로 저장

# DataFrame 생성
df = pd.DataFrame(results)

# 데이터 확인
print(df.head())  # 상위 5개 출력

# 엑셀로 저장
#df.to_excel("/content/drive/MyDrive/Colab Notebooks/RAGNote_project/search_results_ratio0.3.xlsx", index=False)

                                          Query  Similarity  \
0           횡성군 민원행정 서비스 고객만족도 조사의 궁극적인 목표는 뭐야?    0.708568   
1           횡성군 민원행정 서비스 고객만족도 조사의 궁극적인 목표는 뭐야?    0.524660   
2  횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?    0.718936   
3  횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?    0.631480   
4  횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?    0.514028   

                                             Content  
0  연구배경 및 목적\n1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구...  
1  11. 30(35일)\n▷ 공간적 범위\n- 횡성군 종합민원실\n▷ 내용적 범위\n...  
2  연구배경 및 목적\n1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구...  
3  11. 30(35일)\n▷ 공간적 범위\n- 횡성군 종합민원실\n▷ 내용적 범위\n...  
4  발전방안\n1. 만족도 평가와 비교분석을 통한 문제점 탐색\n▷ 2006년 상반기 ...  


In [14]:
# 답변 생성 함수
def answer_print(queries):
  for i, query in enumerate(queries):

      # 해당 query에 대한 문서 리트리브
      retrieved_docs = retrieve_documents_by_similarity_ratio(query, model, texts, threshold_ratio=0.3)

      # context 텍스트 생성
      context_text = "\n".join([doc[0] for doc in retrieved_docs])

      # Gemini API를 통한 응답 생성
      answer = generate_gemini_answer(query, context_text)

      # 출력
      print(f"\n===============================")
      print(f" Query {i+1}: {query}")
      print(f" Gemini 응답: {answer}")

In [15]:
# 답변 생성
answer_print(queries)


 Query 1: 횡성군 민원행정 서비스 고객만족도 조사의 궁극적인 목표는 뭐야?
 Gemini 응답: 횡성군 민원행정 서비스 고객만족도 조사는 민원 서비스 질 향상을 통해 횡성군의 경쟁력을 높이는 것을 궁극적인 목표로 한다. 이를 위해 민원행정 서비스 전달체계, 공무원 태도, 민원 제도, 시설 등에 대한 고객 만족도를 조사하고 분석하여 고객 중심의 행정 서비스를 구현하고, 불합리한 제도를 개선하며, 맞춤형 민원 행정 서비스를 제공하고자 한다. 또한, 전자 민원 행정 구현을 위한 기초 조사를 실시하고, 행정 서비스의 취약점을 파악하여 개선 방향을 도출하며, 과거 만족도 조사 결과와 비교 분석하여 변화 추이를 파악하는 것을 목표로 한다.


 Query 2: 횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?
 Gemini 응답: 횡성군 종합민원실 민원행정서비스 만족도 조사는 고객 중심의 행정 서비스 개선 및 질적 향상을 목표로 실시되었으며, 이를 위해 2명의 설문조사원이 투입되어 1:1 면접 및 자기기입식 설문조사를 진행했습니다.


 Query 3: 횡성군 종합민원실의 민원행정서비스 조사는 며칠을 선정하여 실시하였어?
 Gemini 응답: 횡성군 종합민원실 민원행정서비스 만족도 조사는 고객 중심의 행정 서비스 개선 및 질적 향상을 목표로, 민원 서비스 이용자들을 대상으로 만족도를 측정하여 문제점을 파악하고 개선 방안을 도출하는 것을 목적으로 한다. 조사는 설문 조사를 통해 진행되었으며, 2006년 11월 2일부터 11월 15일까지 공휴일과 휴일을 제외한 **10일** 동안 횡성군 종합민원실에서 실시되었다.


 Query 4: 민원행정서비스에 대한 이용고객의 만족도를 평가하기 위한 설문지의 세부설문은 몇 개의 항목으로 구성되어 있을까?
 Gemini 응답: 횡성군 종합민원실은 민원행정서비스 질적 향상을 위해 고객 만족도 조사를 실시했으며, 설문지는 민원시설, 서비스, 민원처리, 인구통계적 요소로 구성된 총 22개 항목으로 이루어

In [16]:
# 전체 query - response 쌍에 대해 RAGAS 평가 수행

avg_faithfulness, avg_relevancy, avg_precision, avg_recall = 0, 0, 0, 0

for q_idx in range(10):
    # 해당 query에 대한 문서 리트리브
    retrieved_docs = retrieve_documents_by_similarity_ratio(queries[q_idx], model, texts, threshold_ratio=0.3)

    # context 텍스트 생성
    context_text = "\n".join([doc[0] for doc in retrieved_docs])

    # Gemini API를 통한 응답 생성
    answer = generate_gemini_answer(queries[q_idx], context_text)
    
    # 출력
    print(f"\n===============================")
    print(f" Query {i+1}: {queries[q_idx]}")
    print(f" Gemini 응답: {answer}")
    
    # evaluate RAG with metrics of RAGAS

    data = {
        "user_input": [queries[q_idx]],
        "response": [answer],
        "retrieved_contexts": [[context_text]],
        "reference": [responses[q_idx]],
    }

    dataset = Dataset.from_dict(data)

    metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ]

    results = evaluate(dataset, metrics)

    df = results.to_pandas()
    
    print(f"Faithfulness: {df['faithfulness'].item()}")
    print(f"Answer Relevancy: {df['answer_relevancy'].item()}")
    print(f"Context Precision: {df['context_precision'].item()}")
    print(f"Context Recall: {df['context_recall'].item()}")
    
    avg_faithfulness += df['faithfulness'].item()
    avg_relevancy += df['answer_relevancy'].item()
    avg_precision += df['context_precision'].item()
    avg_recall += df['context_recall'].item()

# 평균값 계산
avg_faithfulness /= len(queries)
avg_relevancy /= len(queries)
avg_precision /= len(queries)
avg_recall /= len(queries)

print(f"Average Faithfulness: {avg_faithfulness}")
print(f"Average Answer Relevancy: {avg_relevancy}")
print(f"Average Context Precision: {avg_precision}")
print(f"Average Context Recall: {avg_recall}")


 Query 10: 횡성군 민원행정 서비스 고객만족도 조사의 궁극적인 목표는 뭐야?
 Gemini 응답: 횡성군 민원행정 서비스 고객만족도 조사는 민원행정 서비스의 질을 향상시켜 고객 중심의 행정 서비스를 구현하고, 궁극적으로 횡성군의 경쟁력을 제고하는 것을 목표로 합니다. 이를 위해 민원 서비스 이용자들의 만족도를 조사하여 불합리한 제도를 개선하고, 맞춤형 민원 행정 서비스를 제공하며, 전자민원 행정 구현을 위한 기초 자료를 확보하고자 합니다. 또한, 과거의 고객 만족도 조사 결과와 비교 분석하여 변화 정도를 파악하고, 행정 서비스의 취약점을 개선하는 데 활용합니다.



Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Faithfulness: 1.0
Answer Relevancy: 0.8991614731161803
Context Precision: 0.9999999999
Context Recall: 1.0

 Query 10: 횡성군 종합민원실의 민원 만족도 설문조사를 위해 투입된 설문조사원은 몇 명이야?
 Gemini 응답: 횡성군 종합민원실 민원 만족도 설문조사를 위해 2명의 조사원이 투입되었으며, 이들은 사전 교육을 수료했다. 설문조사는 2006년 11월 2일부터 11월 15일까지 10일간 진행되었고, 횡성군 종합민원실에서 민원인 524명을 대상으로 1:1 면접 또는 자기기입 방식으로 실시되었다.



Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Faithfulness: 1.0
Answer Relevancy: 0.8619147287017531
Context Precision: 0.9999999999
Context Recall: 1.0

 Query 10: 횡성군 종합민원실의 민원행정서비스 조사는 며칠을 선정하여 실시하였어?
 Gemini 응답: 횡성군 종합민원실 민원행정서비스 만족도 조사는 고객 중심의 행정 서비스 개선을 목표로, 민원 서비스 시설, 공무원 친절도, 민원 처리 내용 등을 평가합니다. 2006년 하반기 조사는 상반기 조사 결과와 비교 분석하여 개선점을 도출하고, 고객 만족도 향상을 위한 기초 자료로 활용하고자 했습니다. 이를 위해 **공휴일과 휴일을 제외한 10일**을 선정하여 횡성군 종합민원실을 이용한 민원인 524명을 대상으로 설문 조사를 실시했습니다.



...

Faithfulness: 0.4
Answer Relevancy: 0.8062907229536659
Context Precision: 0.9999999999
Context Recall: 1.0

 Query 10: 회계평가의 세부항목은 몇 개야?
 Gemini 응답: 비영리 민간단체 지원사업 평가는 사업평가와 회계평가로 나뉘어 진행되며, 회계평가 항목은 정산자료 구비상태 등 총 9개 항목으로 구성돼.


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Faithfulness: 1.0
Answer Relevancy: 0.7703091685458228
Context Precision: 0.9999999999
Context Recall: 0.0

 Query 10: 2020년 사업평가 점수가 떨어진 이유는 뭘까?
 Gemini 응답: 2020년 비영리민간단체 공익활동 지원사업 종합평가 결과, 사업평가 점수가 전년 대비 하락했는데, 이는 코로나19 팬데믹으로 인해 대부분의 단체들이 사업 내용이나 방식을 변경해야 했고, 계획된 단위사업을 모두 수행하지 못하여 성과 측면에서 점수가 하락했기 때문입니다.



Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

Faithfulness: 1.0
Answer Relevancy: 0.79151065276489
Context Precision: 0.9999999999
Context Recall: 1.0
Average Faithfulness: 0.8677777777777778
Average Answer Relevancy: 0.66653058531335
Average Context Precision: 0.9999999999
Average Context Recall: 0.8
